In [80]:
import feedparser
from datetime import datetime
from dateutil import tz
from operator import itemgetter
import requests

In [121]:
feeds = [
    {'url': 'https://rss.newsru.com/russia/', 'dt_format': '%a, %d %b %Y %H:%M:%S %z'},
    {'url': 'https://rss.newsru.com/world/', 'dt_format': '%a, %d %b %Y %H:%M:%S %z'},
    {'url': 'https://www.interfax.ru/rss.asp', 'dt_format': '%a, %d %b %Y %H:%M:%S %z'},
    {'url': 'https://ria.ru/export/rss2/index.xml', 'dt_format': '%a, %d %b %Y %H:%M:%S %z'},
    {'url': 'https://news.yandex.ru/society.rss', 'dt_format': '%d %b %Y %H:%M:%S %z'}
]
needles = ['эстони', 'швеци', 'финлянд', 'хельсинк', 'таллин', 'стокгольм', 'корона']
current_tz = tz.gettz('Europe/Moscow')
dt_format = '%H:%M %d.%m'
countries_news = []

for feed in feeds:
    d = feedparser.parse(feed['url'])
    print(feed['url'], len(d['entries']))
    for item in d['entries']:
        if any(needle in item['title'].lower() for needle in needles) or any(needle in item['summary'].lower() for needle in needles):
            dt = datetime.strptime(item['published'], feed['dt_format']).astimezone(tz=current_tz)
            countries_news.append({
                'title': item['title'],
                'link': item['link'],
                'published_origin': item['published'],
                'published': dt.strftime(dt_format),
                'published_dt': dt,
                'diff': (datetime.now(tz=current_tz) - dt).total_seconds() / (60 * 60)
            })


https://rss.newsru.com/russia/ 50
https://rss.newsru.com/world/ 50
https://www.interfax.ru/rss.asp 25
https://ria.ru/export/rss2/index.xml 20
https://news.yandex.ru/society.rss 24


In [104]:
news_feed_url = 'https://tass.ru/live/api/v1/get_feed?timestamp=1684054129&limit=100&slug=obschestvo&theme=%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%BE%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%BA%D0%BE%D1%80%D0%BE%D0%BD%D0%B0%D0%B2%D0%B8%D1%80%D1%83%D1%81%D0%B0%20%D0%BD%D0%BE%D0%B2%D0%BE%D0%B3%D0%BE%20%D1%82%D0%B8%D0%BF%D0%B0'            
tass_news = requests.get(news_feed_url).json().get('data', {}).get('news', [])


In [105]:
tass_news[0]

{'update': True,
 'publish_date': '2020-03-14T03:37:26+03:00',
 'subtitle': '',
 'slug': 'obschestvo',
 'theme': 'Распространение коронавируса нового типа',
 'id': 7977667,
 'title': 'AP: второй человек, посетивший резиденцию Трампа, заражен коронавирусом',
 'is_flash': False,
 'link': 'https://tass.ru/obschestvo/7977667'}

In [122]:
for news_item in tass_news:
    if any(needle in news_item['title'].lower() for needle in needles):
        dt = datetime.fromisoformat(news_item['publish_date']).astimezone(tz=current_tz)
        countries_news.append({
                'title': news_item['title'],
                'link': news_item['link'],
                'published_origin': news_item['publish_date'],
                'published': dt.strftime(dt_format),
                'published_dt': dt,
                'diff': (datetime.now(tz=current_tz) - dt).total_seconds() / (60 * 60)
            })

In [123]:
#     for news_item in news:
#         if any(needle in news_item['title'].lower() for needle in needles):
#             dt = datetime.fromisoformat(news_item['publish_date']).strftime('%H:%M %d.%m')
#             news_block += f"{news_item['title']}\n{dt} | {news_item['link']}\n\n"
            

In [124]:
countries_news = sorted(countries_news, key=itemgetter('published_dt'), reverse=True)

In [125]:
countries_news

[{'title': 'В Рио-де-Жанейро приостановили культурные мероприятия из-за COVID-19',
  'link': 'https://ria.ru/20200314/1568589361.html',
  'published_origin': 'Sat, 14 Mar 2020 03:59:27 +0300',
  'published': '03:59 14.03',
  'published_dt': datetime.datetime(2020, 3, 14, 3, 59, 27, tzinfo=tzfile('/usr/share/zoneinfo/Europe/Moscow')),
  'diff': 0.1011938411111111},
 {'title': 'В венском вузе исключили возможность занижения данных по коронавирусу',
  'link': 'https://ria.ru/20200314/1568589248.html',
  'published_origin': 'Sat, 14 Mar 2020 03:37:51 +0300',
  'published': '03:37 14.03',
  'published_dt': datetime.datetime(2020, 3, 14, 3, 37, 51, tzinfo=tzfile('/usr/share/zoneinfo/Europe/Moscow')),
  'diff': 0.4611939463888889},
 {'title': 'AP: второй человек, посетивший резиденцию Трампа, заражен коронавирусом',
  'link': 'https://tass.ru/obschestvo/7977667',
  'published_origin': '2020-03-14T03:37:26+03:00',
  'published': '03:37 14.03',
  'published_dt': datetime.datetime(2020, 3, 14, 3

In [66]:
# Yandex news
moscow_tz = tz.gettz('Europe/Moscow')
dt = datetime.strptime('13 Mar 2020 19:39:07 +0000', '%d %b %Y %H:%M:%S %z')
dt_moscow = dt.astimezone(tz=moscow_tz)
dt_moscow.strftime('%H:%M %d.%m')

'22:39 13.03'

'22:39 13.03'

In [68]:
# ria, interfax
dt = datetime.strptime('Sat, 14 Mar 2020 01:20:09 +0300', '%a, %d %b %Y %H:%M:%S %z')
dt_moscow = dt.astimezone(tz=moscow_tz)
dt_moscow.strftime('%H:%M %d.%m')

'01:20 14.03'